Імпортуємо необхідні бібліотеки

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score, silhouette_score, adjusted_rand_score


Завантажуємо дані з файлу "data_lab4.xlsx" і обираємо необхідний лист:

In [2]:
data = pd.read_excel("data_lab4.xlsx", sheet_name="Sheet1")

FileNotFoundError: [Errno 2] No such file or directory: 'data_lab4.xlsx'

Виконуємо попередню обробку даних, включаючи видалення пацієнтів з відсутніми замірами

In [ ]:
data = data.dropna()

Розділяємо дані на дві групи залежно від типу досліджень (температура або час)

In [ ]:
temperature_data = data.iloc[:, :5]
time_data = data.iloc[:, 5:]


Побудова графіку паралельних координат для температурних досліджень:

In [ ]:
plt.figure(figsize=(10, 6))
for i in range(len(temperature_data)):
    plt.plot(range(36, 31, -1), temperature_data.iloc[i].values, label="Patient {}".format(i+1))
plt.xlabel("Temperature")
plt.ylabel("Measurement")
plt.title("Parallel Coordinates Plot (Temperature Data)")
plt.legend()
plt.show()


Побудова графіку паралельних координат для досліджень за часом:

In [ ]:
plt.figure(figsize=(10, 6))
for i in range(len(time_data)):
    plt.plot(range(3, 49, 3), time_data.iloc[i].values, label="Patient {}".format(i+1))
plt.xlabel("Time (hours)")
plt.ylabel("Measurement")
plt.title("Parallel Coordinates Plot (Time Data)")
plt.legend()
plt.show()


Кластеризація за допомогою алгоритму k-середніх:

In [ ]:
# Підготовка даних для кластеризації
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)

# Застосування алгоритму k-середніх
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans_labels = kmeans.fit_predict(scaled_data)

# Візуалізація результатів кластеризації на графіку паралельних координат
plt.figure(figsize=(10, 6))
for i in range(len(data)):
    plt.plot(range(1, 17), scaled_data[i], label="Patient {}".format(i+1), color=plt.cm.tab10(kmeans_labels[i]))
plt.xlabel("Measurement")
plt.ylabel("Scaled Value")
plt.title("Parallel Coordinates Plot (k-means Clustering)")
plt.legend()
plt.show()


Кластеризація за допомогою ієрархічних алгоритмів:

In [ ]:
# Агломеративна кластеризація
agg_clustering = AgglomerativeClustering(n_clusters=3)
agg_labels = agg_clustering.fit_predict(scaled_data)

# Дендрограма для агломеративної кластеризації
plt.figure(figsize=(10, 6))
dendrogram(agg_clustering.linkage_, labels=agg_labels)
plt.title("Dendrogram (Agglomerative Clustering)")
plt.xlabel("Patient")
plt.ylabel("Distance")
plt.show()


Оцінка кількості кластерів для різних алгоритмів за допомогою індексів якості:

In [ ]:
# Оцінка для алгоритму k-середніх
ch_score = calinski_harabasz_score(scaled_data, kmeans_labels)
db_score = davies_bouldin_score(scaled_data, kmeans_labels)
silhouette_avg = silhouette_score(scaled_data, kmeans_labels)

# Оцінка для агломеративної кластеризації
ch_score_agg = calinski_harabasz_score(scaled_data, agg_labels)
db_score_agg = davies_bouldin_score(scaled_data, agg_labels)
silhouette_avg_agg = silhouette_score(scaled_data, agg_labels)

print("k-means Clustering:")
print("Calinski-Harabasz Score:", ch_score)
print("Davies-Bouldin Score:", db_score)
print("Silhouette Score:", silhouette_avg)

print("\nAgglomerative Clustering:")
print("Calinski-Harabasz Score:", ch_score_agg)
print("Davies-Bouldin Score:", db_score_agg)
print("Silhouette Score:", silhouette_avg_agg)


Порівняння розбиття за допомогою adjusted Rand Index (ARI):

In [ ]:
kmeans_rand_index = adjusted_rand_score(kmeans_labels, agg_labels)
agg_rand_index = adjusted_rand_score(agg_labels, kmeans_labels)

print("Adjusted Rand Index (k-means vs. Agglomerative Clustering):", kmeans_rand_index)
print("Adjusted Rand Index (Agglomerative Clustering vs. k-means):", agg_rand_index)
